In [ ]:
# ! cp drive/MyDrive/Challenge1.zip ./

In [ ]:
# ! mkdir -p Challenge1
# ! mv Challenge1.zip Challenge1

In [ ]:
# ! cd Challenge1 && unzip Challenge1.zip

In [ ]:
# ! ls ./Challenge1/coin-dataset/ | grep ^_anno

In [1]:
# ! pip install pycocotools mlflow pyngrok

In [ ]:
import mlflow
from pyngrok import ngrok

# Start MLflow server in background
get_ipython().system_raw('mlflow ui --port 5000 &')

ngrok.set_auth_token("2peCOTT2EDUhQ2eQAIods2Lwg60_3QFrEbrgq4tkxc9wLn9Mk")

# Expose port 5000 via ngrok
public_url = ngrok.connect(5000)
print(f"MLflow UI available at: {public_url}")


In [ ]:
import os
import torch
import numpy as np
import random
from torch.utils.data import DataLoader, Subset
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.transforms import functional as F
from pycocotools.coco import COCO
from PIL import Image
import mlflow
import shutil

# -------- Custom COCO Dataset --------
class COCODataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, ann_path, transforms=None):
        self.coco = COCO(ann_path)
        self.img_dir = img_dir
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.transforms = transforms
        self.ann_path = ann_path

    def __getitem__(self, index):
        img_id = self.ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        path = self.coco.loadImgs(img_id)[0]['file_name']

        img = Image.open(os.path.join(self.img_dir, path)).convert('RGB')

        boxes, labels, masks = [], [], []

        for ann in anns:
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])
            mask = self.coco.annToMask(ann) if 'segmentation' in ann and ann['segmentation'] else np.zeros((img.height, img.width), dtype=np.uint8)
            masks.append(mask)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(np.array(masks), dtype=torch.uint8)

        target = {'boxes': boxes, 'labels': labels, 'masks': masks, 'image_id': torch.tensor([img_id])}

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.ids)

def split_dataset(dataset, split_ratio=0.7):
    indices = list(range(len(dataset)))
    random.shuffle(indices)
    split = int(split_ratio * len(dataset))
    return Subset(dataset, indices[:split]), Subset(dataset, indices[split:])

def collate_fn(batch):
    return tuple(zip(*batch))

def evaluate_model(model, val_loader, device):
    model.train()  # Keep model in train mode (required for torchvision detection models)
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            val_loss += losses.item()
    return val_loss / len(val_loader)

def save_sample_images(dataset, indices, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for idx in indices:
        img, _ = dataset[idx]
        img_np = np.transpose(img.numpy(), (1, 2, 0)) * 255
        img_np = img_np.astype(np.uint8)
        Image.fromarray(img_np).save(os.path.join(save_dir, f"sample_{idx}.png"))

def train():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on device: {device}")

    img_dir = '/content/Challenge1/coin-dataset'
    ann_path = '/content/Challenge1/coin-dataset/_annotations.coco.json'

    full_dataset = COCODataset(img_dir, ann_path, transforms=F.to_tensor)
    train_dataset, val_dataset = split_dataset(full_dataset)

    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

    weights = MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1
    model = maskrcnn_resnet50_fpn_v2(weights=weights)

    num_classes = len(full_dataset.coco.getCatIds()) + 1
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    num_epochs = 20
    best_val_loss = float('inf')

    mlflow.set_experiment("CoinDetectionTraining")

    with mlflow.start_run():
        mlflow.log_param("epochs", num_epochs)
        mlflow.log_param("batch_size", 1)
        mlflow.log_param("learning_rate", 0.005)
        mlflow.log_param("split_ratio", 0.7)
        mlflow.log_param("num_classes", num_classes)

        # Log COCO annotations JSON
        if os.path.exists(ann_path):
            mlflow.log_artifact(ann_path, artifact_path="dataset/coco_annotations")

        # Log Train Samples
        train_img_dir = "./mlflow_train_samples"
        train_indices = train_dataset.indices if hasattr(train_dataset, "indices") else range(min(3, len(train_dataset)))
        save_sample_images(full_dataset, train_indices, train_img_dir)
        mlflow.log_artifacts(train_img_dir, artifact_path="dataset/train_samples")
        shutil.rmtree(train_img_dir)

        # Log Val Samples
        val_img_dir = "./mlflow_val_samples"
        val_indices = val_dataset.indices if hasattr(val_dataset, "indices") else range(min(3, len(val_dataset)))
        save_sample_images(full_dataset, val_indices, val_img_dir)
        mlflow.log_artifacts(val_img_dir, artifact_path="dataset/val_samples")
        shutil.rmtree(val_img_dir)

        # -------- Training Loop --------
        for epoch in range(num_epochs):
            model.train()
            total_loss = 0.0

            for images, targets in train_loader:
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())

                optimizer.zero_grad()
                losses.backward()
                optimizer.step()

                total_loss += losses.item()

            avg_train_loss = total_loss / len(train_loader)
            val_loss = evaluate_model(model, val_loader, device)

            mlflow.log_metric("train_loss", avg_train_loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)

            print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), "best_model.pth")
                mlflow.log_artifact("best_model.pth")
                print(f"✅ Saved new best model at epoch {epoch+1} with val_loss: {val_loss:.4f}")

            lr_scheduler.step()

    print("Training Complete ✅")

if __name__ == "__main__":
    train()